## Auto reload

In [1]:
%load_ext autoreload
%autoreload 2

## LIB

In [1]:
%reload_ext autoreload

# Basic lib
import pandas as pd
import numpy as np
import os
import sys

# config
import yaml

# Custom lib
from src.loader import Loader
from src.loan_preprocessor import Loan_Preprocessor
from src.log_preprocessor import Log_Preprocessor
from src.user_preprocessor import User_Preprocessor

## Load Config

In [2]:
with open("config.yaml", 'r') as config_file:
    config_dict = yaml.load(config_file, Loader=yaml.FullLoader)

## Load Data

In [3]:
loader_config = config_dict.get('Loader')

In [4]:
loader_config = config_dict.get('Loader')
loader = Loader(loader_config)
loan_df = loader.run('loan_result.csv')
log_df = loader.run('log_data.csv')
user_df = loader.run('user_spec.csv')

## Preprocess

In [5]:
prep_config = config_dict.get('Preprocessor')

# Loan preprocess
loan_preprocessor = Loan_Preprocessor(loan_df, prep_config)
prep_loan_df = loan_preprocessor.run(
    save_file_name='prep_loan.fth',
    save_mode=True
    )

필요 없는 열 삭제 중...
datetime으로 바꾸는 중...
결측치를 가지는 행 삭제 중...
✅ prep dataset saved at (prep\prep_loan.fth)


In [6]:
# Log preprocess
log_preprocessor = Log_Preprocessor(log_df, prep_config)
prep_log_df = log_preprocessor.run(
    save_file_name='prep_log.fth',
    save_mode=True
)

필요 없는 열 삭제 중...
datetime으로 바꾸는 중...
카테고리화 시키는 중...
시간축을 기준으로 정렬 중...
✅ prep dataset saved at (prep\prep_log.fth)


In [7]:
# User preprocess
user_preprocessor = User_Preprocessor(user_df, prep_config)
prep_user_df = user_preprocessor.run(
    save_file_name='prep_user.fth',
    save_mode=True
)

datetime으로 바꾸는 중...
파생변수 생성 중...
카테고리화 시키는 중...
원핫인코딩 중...
순서형인코딩 중...
✅ prep dataset saved at (prep\prep_user.fth)


## Matcher

In [10]:
from src.matcher import Matcher
matcher_config = config_dict.get('Matcher')

In [11]:
matcher_config

{'split_year': 2022, 'split_month': 6, 'split_day': 1}

In [14]:
matcher = Matcher(prep_user_df, prep_loan_df, prep_log_df, matcher_config)

In [ ]:
train, test = matcher.run()

## Modeling

In [ ]:
# base line
# model_name : RF, XGB, LGBM, 

## 모델 서치 ##
# from src.models import XGB
# random_state = 42
# xgb = XGB(train_X, train_Y, val_X, val_Y, test_X, test_Y, random_state)
# best_model = xgb.grid_search()

## test 결과 확인 및 모델 해석 ##
# xgb.test_score(best_model)
# xgb.confusion_matrix(best_model)
# xgb.feature_importance(best_model)
# xgb.shap(best_model)


## Inference

## Evaluation